In [3]:
from openwind.sar_data import fetch_asf_s1_data, preprocess_s1_data
import asf_search as asf
from shapely.geometry import Polygon
import folium

### Search and download Sentinel-1 data
1. Specify search parameters such as area, platform, product type, time start, and time end, etc.
2. Find data at the Alaska SAR facility (ASF) using **asf_search** (https://docs.asf.alaska.edu/asf_search/basics/) package and src parameters
3. Explore data on an interactive map
4. Download products from ASF to a local directory

In [4]:
# Step 1: Define an area for the data search, for example by defining corners of a rectangular polygon.
# API will find all data in ASF database overlaping with the search area 
# NOTE: Any polygone shape can be used to search for the data
src_poly = Polygon([(16, -31.5), (20, -31.5), (20, -36), (16, -36), (16, -31.5)])

# Step 2: Define search parameters such as platform, beam, time coverage, and product type.
# More information on available query fields can be found at https://docs.asf.alaska.edu/asf_search/searching/
# The openwind requires a query dictionary:

query = {
    'platform': asf.PLATFORM.SENTINEL1,
    'start': '2022-01-10T00:00:00Z',
    'end': '2022-01-10T23:59:59Z',
    'beamMode': asf.BEAMMODE.IW,
    'processingLevel': asf.PRODUCT_TYPE.GRD_HD,
    'intersectsWith': src_poly.wkt,   # NOTE: We add a geometry from the Step 1 in the WKT format
}

# The list of parameters defined in the <query> will find all Sentinel-1 A and B  Interferometric Wide (IW) 
# Level 1 GRD HD products overlaping with the search polygon on 10 January 2022 between 00:00 and 23:59 hours

In [5]:
# Step 3: Find data at the ASF database according to your quety using openwind fetch_asf_s1_data method
# At this step we only acquire metadata and don't download any data. It saves a lot of time for prelimenary
# data examination.
# NOTE: All found datasets can be downloaded later see Step 3*
asf_query_set, downloaded_uris = fetch_asf_s1_data(query=query, download=False)

# Step 3*. If you want to download all found files you can use
# asf_query_set, downloaded_uris = fetch_asf_s1_data(query=query, download=True, dst='/path/to/data', unpack=True)

>> Found 4 products in ASF database


In [6]:
# Step 4: Plot footprints of downloaded products at an interactive map
# NOTE: You can click of each tile to discover the name of product that can be further used for
# downloading an exact tile (instead of the whole query set)
f = folium.Figure(width=800, height=500)
m = folium.Map(location=[-35, 20], zoom_start=5, max_bounds=True).add_to(f)
for product in asf_query_set:
    f = folium.GeoJson(product.geojson())
    f.add_child(folium.Popup(product.meta['native-id']))
    m.add_child(f)
m

In [7]:
# Step 5: Sentinel-1 metadata from ASF contain a ling for a false color qucklook image which can be
# directly displaied in jupyter notebook.

from IPython.display import Image
print(asf_query_set[0].properties['fileID'])
Image(url=asf_query_set[0].properties['browse'][0], width=800) 

S1A_IW_GRDH_1SDV_20220110T173454_20220110T173519_041401_04EC3B_8986-GRD_HD


In [8]:
# Step 6: Download the data. It is possible to download all granules from the query set or a single granule.
# NOTE: To download data from the ASF you have to create an account at ASF
# We will use the same function as for data searching, but with additional parameters required
# for data download

# Step 6.1. If you don't want to download all products from the query set you can download a single granule
# which can be identified from the search (see Sep 3 and 4)
granule_id = 'S1A_IW_GRDH_1SDV_20220110T173429_20220110T173454_041401_04EC3B_31E4-GRD_HD'
# Step 6.2. Define the path where downloaded product will be stored
dst_path = '/src/.devcontainer/'
asf_query_set, downloaded_uris = fetch_asf_s1_data(
    granule_id=granule_id, # Specific granule ID to be downloaded
    download=True,         # Find and download the data
    dst=dst_path,          # Download data to this path
    unpack=True            # Unpack SAFE product from the zip archive and delete zip archive
)

>> Found 1 products in ASF database
>> Downloading 1 files: 

/opt/conda/lib/python3.7/site-packages/asf_search/download/download.py:62: UserWarning: File already exists, skipping download: /src/.devcontainer/S1A_IW_GRDH_1SDV_20220110T173429_20220110T173454_041401_04EC3B_31E4.zip
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')


0:00:04.453920 complete!
>> Unpacking /src/.devcontainer/S1A_IW_GRDH_1SDV_20220110T173429_20220110T173454_041401_04EC3B_31E4.zip 0:03:37.913016 complete!
